In [1]:
import jax.numpy as jnp
import numpy as np
import pandas as pd

import pykalman
import seaborn as sns
import matplotlib.pyplot as plt
import jax.scipy.special
from functools import partial
from jax.scipy.stats import nbinom

from tqdm import tqdm 
from pfilter_complete import _pfilter_internal_complete

/var/folders/5v/yphhcgv17dz3dwr2sc43_pwm0000gn/T/ipykernel_2601/2253030746.py:10: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


LG model for testing pfilter_complete

In [2]:
mesocosm_data = pd.read_excel("./data/Mesocosmdata.xls", sheet_name=0)

key = jax.random.PRNGKey(2468)  

dentNoPara = mesocosm_data.iloc[:100][['rep', 'day', 'dent.adult']]

dentNoPara['day'] = (dentNoPara['day'] - 1) * 5 + 7

dentNoPara = dentNoPara.iloc[::-1].reset_index(drop=True)

data = []
dentadult = []
trails = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J"]

for trail in trails:
    subset_data = dentNoPara[dentNoPara['rep'] == trail][['day', 'dent.adult']]
    subset_dentadult = dentNoPara[dentNoPara['rep'] == trail][['dent.adult']]
    data.append(subset_data)
    dentadult.append(subset_dentadult)

def transform_thetas(sigSn, sigF, f_Sn, rn, k_Sn, sigJn, theta_Sn, theta_Jn, lambda_Jn):
    return jnp.array([jnp.log(sigSn), jnp.log(sigF), jnp.log(f_Sn), jnp.log(rn), jnp.log(k_Sn), jnp.log(sigJn), jnp.log(theta_Sn), jnp.log(theta_Jn), jnp.log(lambda_Jn)])

def get_thetas(thetas):
    sigSn = jnp.exp(thetas[0])
    sigF = jnp.exp(thetas[1])
    f_Sn = jnp.exp(thetas[2])
    rn = jnp.exp(thetas[3])
    k_Sn = jnp.exp(thetas[4])
    sigJn = jnp.exp(thetas[5])
    theta_Sn = jnp.exp(thetas[6])
    theta_Jn = jnp.exp(thetas[7])
    lambda_Jn = jnp.exp(thetas[8])

    return sigSn, sigF, f_Sn, rn, k_Sn, sigJn, theta_Sn, theta_Jn, lambda_Jn
#state : "Sn","Jn" ,"error_count", "F", "T_Sn", "day_index"
#thetas :  sigSn, sigF, f_Sn, rn, k_Sn, sigJn, theta_Sn, theta_Jn, lambda_Jn

def rproc_loop_noi_rm(state, thetas, key, num_steps):
    # extract states and thetas
    Sn, Jn, F, T_Sn, error_count = state[0], state[1], state[2], state[3], state[4]
    sigSn, sigF, f_Sn, rn, k_Sn, sigJn, theta_Sn, theta_Jn, lambda_Jn = get_thetas(thetas)

    # contant
    delta = 0.013
    dt = 0.25
    
    def loop_body(i, loop_state):
        Sn, Jn, F, T_Sn, error_count, key = loop_state
        # progressing the states and updating them
        Sn_term = lambda_Jn * Jn * dt - theta_Sn * Sn * dt - delta * Sn * dt
        Jn_term = rn * f_Sn * F * Sn * dt - lambda_Jn * Jn * dt - theta_Jn * Jn * dt - delta * Jn * dt
        F_term = -f_Sn * F * (Sn + 1 * Jn) * dt - delta * F * dt + 0.37 * dt

        # keep updating the states
        Sn = Sn_term + Sn
        Jn = Jn_term + Jn
        F = F_term + F

        # extreme cases
        Sn = jnp.where((Sn < 0.0) | (Sn > 1e5), 0.0, Sn)
        error_count += jnp.where((Sn < 0.0) | (Sn > 1e5), 1, 0)

        F = jnp.where((F < 0.0) | (F > 1e20), 0.0, F)
        error_count += jnp.where((F < 0.0) | (F > 1e20), 1000, 0)

        Jn = jnp.where((Jn < 0.0) | (Jn > 1e5), 0.0, Jn)
        error_count += jnp.where((Jn < 0.0) | (Jn > 1e5), 0.001, 0)

        T_Sn = jnp.abs(Sn)

        return Sn, Jn, F, T_Sn, error_count, key

    Sn, Jn, F, T_Sn, error_count, key = jax.lax.fori_loop(0, num_steps, loop_body, (Sn, Jn, F, T_Sn, error_count, key))
    
    return Sn, Jn, F, T_Sn, error_count, key
    
def rproc_loop(state, thetas, key, num_steps):

    Sn, Jn, F, T_Sn, error_count = state[0], state[1], state[2], state[3], state[4]
    sigSn, sigF, f_Sn, rn, k_Sn, sigJn, theta_Sn, theta_Jn, lambda_Jn = get_thetas(thetas)
    
    def loop_body_2(i, loop_state):
        Sn, Jn, F, T_Sn, error_count, main_key, sigSn, sigF, sigJn = loop_state
        dt = 0.25
        delta = 0.013
       
        main_key, key1, key2, key3 = jax.random.split(main_key, 4)
        
        noiSn = jax.random.normal(key = key1) * sigSn * jnp.sqrt(dt) 
        noiF = jax.random.normal(key = key2) * sigF * jnp.sqrt(dt) 
        noiJn = jax.random.normal(key = key3) * sigJn* jnp.sqrt(dt) 

        Sn_term = lambda_Jn * Jn * dt - theta_Sn * Sn * dt - delta * Sn * dt + Sn * noiSn
        Jn_term = rn * f_Sn * F * Sn * dt - lambda_Jn * Jn * dt - theta_Jn * Jn * dt - delta * Jn * dt + Jn * noiJn
        F_term = F * noiF - f_Sn * F * (Sn + 1 * Jn) * dt - delta * F * dt + 0.37 * dt

        Sn = Sn_term + Sn
        Jn = Jn_term + Jn
        F = F_term + F

        Sn = jnp.where((Sn < 0.0) | (Sn > 1e5), 0.0, Sn)
        error_count += jnp.where((Sn < 0.0) | (Sn > 1e5), 1, 0)

        F = jnp.where((F < 0.0) | (F > 1e20), 0.0, F)
        error_count += jnp.where((F < 0.0) | (F > 1e20), 1000, 0)

        Jn = jnp.where((Jn < 0.0) | (Jn > 1e5), 0.0, Jn)
        error_count += jnp.where((Jn < 0.0) | (Jn > 1e5), 0.001, 0)

        T_Sn = jnp.abs(Sn)
        return Sn, Jn, F, T_Sn, error_count, main_key, sigSn, sigF, sigJn

    Sn, Jn, F, T_Sn, error_count, key, sigSn, sigF, sigJn = jax.lax.fori_loop(0, num_steps, loop_body_2, (Sn, Jn, F, T_Sn, error_count, key, sigSn, sigF, sigJn))
    
    return Sn, Jn, F, T_Sn, error_count, key


def rproc_noi_rm(state, thetas, key, covars = None):
    # extract states and thetas
    day_index = state[5]
    int_day_index = day_index.astype(jnp.int32)
    dt = 0.25
    day_diff = covars[int_day_index]
    loop_num = day_diff/dt
    int_loop_num = loop_num.astype(jnp.int32)

    Sn, Jn, F, T_Sn, error_count, key = rproc_loop_noi_rm(state, thetas, key, int_loop_num)

    day_index = day_index + 1
    
    return jnp.array([Sn, Jn, F, T_Sn, error_count, day_index])


def rproc(state, thetas, key, covars = None):

    day_index = state[5]
    int_day_index = day_index.astype(jnp.int32)
    dt = 0.25
    day_diff = covars[int_day_index]
    loop_num = day_diff/dt
    int_loop_num = loop_num.astype(jnp.int32)

    Sn, Jn, F, T_Sn, error_count, key = rproc_loop(state, thetas, key, int_loop_num)

    day_index = day_index + 1
    
    return jnp.array([Sn, Jn, F, T_Sn, error_count, day_index])

rprocess_noi_rm = jax.vmap(rproc_noi_rm, (0, None, 0, None))
rprocesses_noi_rm = jax.vmap(rproc_noi_rm, (0 , 0, 0, None))

rprocess = jax.vmap(rproc, (0, None, 0, None))
rprocesses = jax.vmap(rproc, (0, None, 0, None))

def dnbinom_mu(y_val, k_Sn, T_Sn):
    p = k_Sn/(k_Sn + T_Sn)
    #logpmf(k, n, p, loc=0): n - number of success, p - probability of success
    return nbinom.logpmf(y_val, k_Sn, p)

def dmeas(y_val, state_preds, thetas):
    k_Sn = get_thetas(thetas)[4]
    T_Sn = state_preds[3]
    error_count = state_preds[4]
    log_lik = dnbinom_mu(y_val, k_Sn, T_Sn)
    log_lik_val = jnp.where(error_count>0, -150, log_lik)

    return log_lik_val

dmeasure = jax.vmap(dmeas, (None, 0, None))
dmeasures = jax.vmap(dmeas, (None, 0, 0))

def rinit(thetas, J, covars = None):
    Sn = 3
    F = 16.667
    Jn = 0
    T_Sn = 0.0
    error_count = 0.0

    day_index = 0

    return jnp.tile(jnp.array([Sn, Jn, F, T_Sn, error_count, day_index]), (J, 1))

In [11]:
# create the POMP object with rproc_noi_rm (without randomness)
dentadult1 = jnp.array(data[0]['dent.adult'].to_numpy())
day1 = jnp.array(data[0]['day'].to_numpy())
covars1 = jnp.diff(day1, prepend=0)
        
thetas = transform_thetas(0.1, 0.1, 0.1, 10, 1, 0.1, 0.1, 0.1, 0.1)

neg_loglik, neg_loglik_mean, cond_logliks, particles, filter_mean_arr, ess_arr, filt_traj = _pfilter_internal_complete(theta=thetas, ys=dentadult1, J = 1, rinit = rinit, rprocess = rprocess_noi_rm, dmeasure = dmeasure, covars = covars1, thresh = -1, key = jax.random.PRNGKey(112))

R result:
logLik(pfilter(pomplist_rm[[1]],Np=10)): -147.0411

In [13]:
# negative loglikelihood
print("negative loglik:", neg_loglik)

# mean negative loglike
print("mean negative loglik:", neg_loglik_mean)
print(neg_loglik/len(dentadult1))

negative loglik: 147.04114
mean negative loglik: 14.704114
14.704114


R result:
cond_logLik(pfilter(pomplist_rm[[1]],Np=10)):

 [1]  -3.169481  -3.351047  -6.718464 -16.764215 -32.472745  -7.656859 -16.937234 -11.854782 -27.378105 -20.738201


In [12]:
# conditional loglikelihood 
print("conditional loglikelihood:", cond_logliks)

conditional loglikelihood: [ -3.169481   -3.3510475  -6.718464  -16.764214  -32.472748   -7.6568584
 -16.937235  -11.854784  -27.378105  -20.7382   ]


In [14]:
# state
print("particles:")
print(particles)

particles:
[[[10.468981   12.76933     0.15531066 10.468981    0.
    1.        ]]

 [[10.105731    9.836075    0.18236989 10.105731    0.
    2.        ]]

 [[ 9.314947    9.077141    0.19837272  9.314947    0.
    3.        ]]

 [[ 8.685558    8.760315    0.20944625  8.685558    0.
    4.        ]]

 [[ 8.236955    8.566329    0.21767184  8.236955    0.
    5.        ]]

 [[ 7.921958    8.431014    0.22383532  7.921958    0.
    6.        ]]

 [[ 7.700304    8.333992    0.22841781  7.700304    0.
    7.        ]]

 [[ 7.543664    8.26417     0.23179367  7.543664    0.
    8.        ]]

 [[ 7.432563    8.213955    0.23426199  7.432563    0.
    9.        ]]

 [[ 7.353543    8.177877    0.23605663  7.353543    0.
   10.        ]]]


R result filter_mean(pfilter(pomplist_rm[[1]],Np=100, filter.mean=TRUE)):

$$
\begin{array}{c|cccccccccc}
\textbf{name} & \textbf{1} & \textbf{2} & \textbf{3} & \textbf{4} & \textbf{5} & \textbf{6} & \textbf{7} & \textbf{8} & \textbf{9} & \textbf{10} \\
\hline
\text{Sn} & 10.4689787 & 10.1057278 & 9.3149447 & 8.6855567 & 8.2369538 & 7.9219585 & 7.7003044 & 7.5436637 & 7.432562 & 7.3535427 \\
\text{Jn} & 12.7693270 & 9.8360732 & 9.0771377 & 8.7603146 & 8.5663294 & 8.4310137 & 8.3339900 & 8.2641687 & 8.213954 & 8.1778783 \\
\text{error\_count} & 0.0000000 & 0.0000000 & 0.0000000 & 0.0000000 & 0.0000000 & 0.0000000 & 0.0000000 & 0.0000000 & 0.000000 & 0.0000000 \\
\text{F} & 0.1553107 & 0.1823699 & 0.1983728 & 0.2094463 & 0.2176719 & 0.2238353 & 0.2284178 & 0.2317937 & 0.234262 & 0.2360566 \\
\text{T\_Sn} & 10.4689787 & 10.1057278 & 9.3149447 & 8.6855567 & 8.2369538 & 7.9219585 & 7.7003044 & 7.5436637 & 7.432562 & 7.3535427 \\
\end{array}
$$


In [15]:
# filter mean
print("filter mean", filter_mean_arr)


filter mean [[10.468981   12.76933     0.15531066 10.468981    0.          1.        ]
 [10.105731    9.836075    0.18236989 10.105731    0.          2.        ]
 [ 9.314947    9.077141    0.19837272  9.314947    0.          3.        ]
 [ 8.685558    8.760315    0.20944625  8.685558    0.          4.        ]
 [ 8.236955    8.566329    0.21767184  8.236955    0.          5.        ]
 [ 7.921958    8.431014    0.22383532  7.921958    0.          6.        ]
 [ 7.700304    8.333992    0.22841781  7.700304    0.          7.        ]
 [ 7.543664    8.26417     0.23179367  7.543664    0.          8.        ]
 [ 7.432563    8.213955    0.23426199  7.432563    0.          9.        ]
 [ 7.353543    8.177877    0.23605663  7.353543    0.         10.        ]]


R result: filter_traj(pfilter(pomplist_rm[[1]],Np=100, filter.traj=TRUE)):

1
  Sn           3.000
  Jn           0.000
  error_count  0.000
  F           16.667
  T_Sn         0.000

2

  Sn          10.4689787
  Jn          12.7693270
  error_count  0.0000000
  F            0.1553107
  T_Sn        10.4689787

3

  Sn          10.1057278
  Jn           9.8360732
  error_count  0.0000000
  F            0.1823699
  T_Sn        10.1057278

4

  Sn          9.3149447
  Jn          9.0771377
  error_count 0.0000000
  F           0.1983728
  T_Sn        9.3149447

5

  Sn          8.6855567
  Jn          8.7603146
  error_count 0.0000000
  F           0.2094463
  T_Sn        8.6855567

6

  Sn          8.2369538
  Jn          8.5663294
  error_count 0.0000000
  F           0.2176719
  T_Sn        8.2369538

7

  Sn          7.9219585
  Jn          8.4310137
  error_count 0.0000000
  F           0.2238353
  T_Sn        7.9219585

8

  Sn          7.7003044
  Jn          8.3339900
  error_count 0.0000000
  F           0.2284178
  T_Sn        7.7003044

9

  Sn          7.5436637
  Jn          8.2641687
  error_count 0.0000000
  F           0.2317937
  T_Sn        7.5436637

10

  Sn          7.432562
  Jn          8.213954
  error_count 0.000000
  F           0.234262
  T_Sn        7.432562

11

  Sn          7.3535427
  Jn          8.1778783
  error_count 0.0000000
  F           0.2360566
  T_Sn        7.3535427



In [17]:
# filter traj
print("filter.traj", filt_traj)

filter.traj [[10.468981   12.76933     0.15531066 10.468981    0.          1.        ]
 [10.105731    9.836075    0.18236989 10.105731    0.          2.        ]
 [ 9.314947    9.077141    0.19837272  9.314947    0.          3.        ]
 [ 8.685558    8.760315    0.20944625  8.685558    0.          4.        ]
 [ 8.236955    8.566329    0.21767184  8.236955    0.          5.        ]
 [ 7.921958    8.431014    0.22383532  7.921958    0.          6.        ]
 [ 7.700304    8.333992    0.22841781  7.700304    0.          7.        ]
 [ 7.543664    8.26417     0.23179367  7.543664    0.          8.        ]
 [ 7.432563    8.213955    0.23426199  7.432563    0.          9.        ]
 [ 7.353543    8.177877    0.23605663  7.353543    0.         10.        ]]
